In [1]:
import sys
sys.path.append('../src')

In [2]:
import pickle
import os
import collections
import numpy as np
import math
import encoder_decoder_uttt as ed
from utictactoe_board import board as c_board
import copy
import torch
import torch.multiprocessing as mp
from alpha_net_uttt import ConnectNet
import datetime
import logging
from tqdm import tqdm

In [3]:
net = ConnectNet()

In [4]:
cuda = torch.cuda.is_available()

In [5]:
net.cuda()

ConnectNet(
  (conv): ConvBlock(
    (conv1): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (res_0): ResBlock(
    (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (res_1): ResBlock(
    (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  

In [6]:
with torch.no_grad():
    current_board = c_board()
    checkmate = False
    board_state = copy.deepcopy(ed.encode_board(current_board))

In [11]:
root = UCTNode(current_board, move=None, parent=DummyNode())

In [12]:
leaf = root.select_leaf()

In [12]:
encoded_s = ed.encode_board(leaf.game)
encoded_s = encoded_s.transpose(2,0,1)
encoded_s = torch.from_numpy(encoded_s).float().cuda()

In [13]:
child_priors, value_estimate = net(encoded_s)

In [14]:
child_priors

tensor([[0.0193, 0.0040, 0.0075, 0.0174, 0.0051, 0.0087, 0.0133, 0.0299, 0.0164,
         0.0120, 0.0039, 0.0099, 0.0259, 0.0093, 0.0139, 0.0290, 0.0123, 0.0220,
         0.0064, 0.0097, 0.0107, 0.0080, 0.0092, 0.0080, 0.0113, 0.0185, 0.0071,
         0.0173, 0.0112, 0.0144, 0.0077, 0.0039, 0.0087, 0.0095, 0.0082, 0.0142,
         0.0167, 0.0056, 0.0119, 0.0195, 0.0044, 0.0109, 0.0115, 0.0160, 0.0072,
         0.0120, 0.0191, 0.0163, 0.0083, 0.0083, 0.0135, 0.0130, 0.0099, 0.0139,
         0.0086, 0.0081, 0.0074, 0.0119, 0.0296, 0.0076, 0.0172, 0.0145, 0.0085,
         0.0127, 0.0086, 0.0097, 0.0069, 0.0063, 0.0101, 0.0153, 0.0282, 0.0142,
         0.0121, 0.0169, 0.0261, 0.0104, 0.0052, 0.0106, 0.0083, 0.0127, 0.0103]],
       device='cuda:0', grad_fn=<ExpBackward>)

In [15]:
value_estimate

tensor([[0.2477]], device='cuda:0', grad_fn=<TanhBackward>)

In [16]:
child_priors = child_priors.detach().cpu().numpy().reshape(-1)
value_estimate = value_estimate.item()

In [17]:
child_priors

array([0.01931141, 0.00399547, 0.00754106, 0.01741103, 0.00509694,
       0.00869306, 0.01329077, 0.02989013, 0.01640395, 0.01200403,
       0.00390759, 0.00992669, 0.0258514 , 0.00934633, 0.01393697,
       0.02898722, 0.0123076 , 0.02197939, 0.00641661, 0.00969115,
       0.01065872, 0.00801833, 0.00917212, 0.00803286, 0.0113274 ,
       0.01853922, 0.00705141, 0.01734105, 0.01123265, 0.0144056 ,
       0.00773642, 0.00387887, 0.00874227, 0.00949395, 0.00824089,
       0.01415952, 0.0166562 , 0.00564346, 0.01187098, 0.01945696,
       0.00435015, 0.01088819, 0.01153302, 0.01601823, 0.00722278,
       0.01196013, 0.01907713, 0.01626409, 0.00827421, 0.00831893,
       0.01348134, 0.0130278 , 0.00994574, 0.01394385, 0.00859241,
       0.00809268, 0.00742167, 0.01192899, 0.0295674 , 0.00762918,
       0.01720848, 0.01454355, 0.00845362, 0.01271035, 0.00860583,
       0.00968458, 0.00687341, 0.00628944, 0.01008539, 0.01526313,
       0.02819048, 0.01423196, 0.01209819, 0.01694877, 0.02607

In [13]:
leaf.game.check_winner()

winner is  None


True

In [16]:
def return_winner(winner):
    if winner == "O" or "X":
        print(1)
        return True
    elif winner == "tie":
        print(2)
        return winner
    else:
        return False

In [17]:
return_winner(winner=None)

1


True

In [30]:
leaf.game.mega_board

array([[' ', ' ', ' '],
       [' ', ' ', ' '],
       [' ', ' ', ' ']], dtype='<U32')

In [20]:
leaf.game.current_board

array([[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']], dtype='<U32')

In [23]:
y = check_ninebox(leaf.game.mega_board)

In [24]:
y

In [21]:
def check_ninebox(mat):
    """
    This function can be use to check the small nine-box \
            or the large nine-box.
    input: mat: a matrix of 3x3
    """
    # look for rows, columns, diagnols
    for i in range(3):
        if mat[i,0] == mat[i,1] == mat[i,2] != ' ':
            return mat[i,0]
        if mat[0,i] == mat[1,i] == mat[2,i] != ' ':
            return mat[0,i]
    if mat[0,0] == mat[1,1] == mat[2,2] != ' ':
        return mat[0,0]
    if mat[0,2] == mat[1,1] == mat[2,0] != ' ':
        return mat[1,1]

    # look for tie
    count = 0
    for i in range(3):
        for j in range(3):
            if mat[i,j] != ' ':
                count += 1
    if count == 9:
        return 'tie'
    else:
        return None

In [9]:
class UCTNode():
    def __init__(self, game, move, parent=None):
        self.game = game # state s
        self.move = move # action index
        self.is_expanded = False
        self.parent = parent  
        self.children = {}
        self.child_priors = np.zeros([81], dtype=np.float32)
        self.child_total_value = np.zeros([81], dtype=np.float32)
        self.child_number_visits = np.zeros([81], dtype=np.float32)
        self.action_idxes = []
        
    @property
    def number_visits(self):
        return self.parent.child_number_visits[self.move]

    @number_visits.setter
    def number_visits(self, value):
        self.parent.child_number_visits[self.move] = value
    
    @property
    def total_value(self):
        return self.parent.child_total_value[self.move]
    
    @total_value.setter
    def total_value(self, value):
        self.parent.child_total_value[self.move] = value
    
    def child_Q(self):
        return self.child_total_value / (1 + self.child_number_visits)
    
    def child_U(self):
        return math.sqrt(self.number_visits) * (
            abs(self.child_priors) / (1 + self.child_number_visits))
    
    def best_child(self):
        if self.action_idxes != []:
            bestmove = self.child_Q() + self.child_U()
            bestmove = self.action_idxes[np.argmax(bestmove[self.action_idxes])]
        else:
            bestmove = np.argmax(self.child_Q() + self.child_U())
        return bestmove
    
    def select_leaf(self):
        current = self
        while current.is_expanded:
            best_move = current.best_child()
            current = current.maybe_add_child(best_move)
        return current
    
    def add_dirichlet_noise(self,action_idxs,child_priors):
        valid_child_priors = child_priors[action_idxs] # select only legal moves entries in child_priors array
        valid_child_priors = 0.75*valid_child_priors + 0.25*np.random.dirichlet(np.zeros([len(valid_child_priors)], \
                                                                                          dtype=np.float32)+192)
        child_priors[action_idxs] = valid_child_priors
        return child_priors
    
    def expand(self, child_priors):
        self.is_expanded = True
        action_idxs = self.game.actions()
        c_p = child_priors
        if action_idxs == []:
            self.is_expanded = False
        self.action_idxes = action_idxs
        c_p[[i for i in range(len(child_priors)) if i not in action_idxs]] = 0.000000000 # mask all illegal actions
        if self.parent.parent == None: # add dirichlet noise to child_priors in root node
            c_p = self.add_dirichlet_noise(action_idxs,c_p)
        self.child_priors = c_p
    
    def decode_n_move_pieces(self,board,move):
        board.drop_piece(move)
        return board
            
    def maybe_add_child(self, move):
        if move not in self.children:
            copy_board = copy.deepcopy(self.game) # make copy of board
            copy_board = self.decode_n_move_pieces(copy_board,move)
            self.children[move] = UCTNode(
              copy_board, move, parent=self)
        return self.children[move]
    
    def backup(self, value_estimate: float):
        current = self
        while current.parent is not None:
            current.number_visits += 1
            if current.game.player == 1: # same as current.parent.game.player = 0
                current.total_value += (1*value_estimate) # value estimate +1 = O wins
            elif current.game.player == 0: # same as current.parent.game.player = 1
                current.total_value += (-1*value_estimate)
            current = current.parent

In [10]:
class DummyNode(object):
    def __init__(self):
        self.parent = None
        self.child_total_value = collections.defaultdict(float)
        self.child_number_visits = collections.defaultdict(float)